In [91]:
# Automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import model_setup as ms
c = ms.ModelConfig('./config/UKR.toml')
# c = ms.ModelConfig('./config/CHN_2024_EFC.toml')

from model_setup.solution_index import SolutionIndexCreator
from model_setup.capacity_setup import CapacitySetup
from model_setup.transmission_setup import TransmissionSetup



In [169]:
ts = TransmissionSetup(c)
ts.calculate_tx_capacities()

In [171]:
ts.capacity_by_border

voltage,220,330,350,400,450,500,750,800
CR-ER,0.000000,405.046283,0.0,0.0,0.0,0.0,2207.5,0.0
CR-NR,0.000000,810.092567,0.0,0.0,0.0,0.0,0.0,0.0
CR-SWR,0.000000,810.092567,0.0,0.0,0.0,0.0,6622.5,0.0
CR-WR,0.000000,0.000000,0.0,0.0,0.0,0.0,2207.5,0.0
CRM-SR,133.913043,1215.138850,0.0,0.0,0.0,0.0,0.0,0.0
EOR-ER,0.000000,810.092567,0.0,0.0,0.0,0.0,6622.5,0.0
EOR-NR,0.000000,1215.138850,0.0,0.0,0.0,0.0,0.0,0.0
ER-NR,0.000000,1620.185133,0.0,0.0,0.0,0.0,0.0,0.0
ER-SR,0.000000,3645.416550,0.0,0.0,0.0,0.0,6622.5,0.0
ER-SWR,0.000000,405.046283,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
border_names = ts.border_df.name.unique()
lines_by_border = ts.gdf_tx.copy()
lines_by_border

lines_by_border.index = lines_by_border.index.astype(int)


In [163]:
sil_by_voltage = ts.df_tx_cap_by_v.set_index('Voltage')['SIL']
sil_by_voltage.loc[lines_by_border.index.astype(int)]*lines_by_border.T


voltage,220,330,350,400,450,500,750,800
CR-ER,0.000000,405.046283,0.0,0.0,0.0,0.0,2207.5,0.0
CR-NR,0.000000,810.092567,0.0,0.0,0.0,0.0,0.0,0.0
CR-SWR,0.000000,810.092567,0.0,0.0,0.0,0.0,6622.5,0.0
CR-WR,0.000000,0.000000,0.0,0.0,0.0,0.0,2207.5,0.0
CRM-SR,133.913043,1215.138850,0.0,0.0,0.0,0.0,0.0,0.0
EOR-ER,0.000000,810.092567,0.0,0.0,0.0,0.0,6622.5,0.0
EOR-NR,0.000000,1215.138850,0.0,0.0,0.0,0.0,0.0,0.0
ER-NR,0.000000,1620.185133,0.0,0.0,0.0,0.0,0.0,0.0
ER-SR,0.000000,3645.416550,0.0,0.0,0.0,0.0,6622.5,0.0
ER-SWR,0.000000,405.046283,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cs.setup_capacity()
cs.setup_plant_parameters()
cs.export_plant_parameters()